The tools in `starlab` are designed to be executed from the command line, and chained together to form simulation pipelines. This notebook walks through the process of executing some `starlab` commands up to and including the type of pipelines needed to run full simulations.

### First steps

In ipython, when you put an ! at the beginning of a cell, the cell's contents get passed to the underlying operating system.  This makes it possible to run starlab commands from within ipython.  For example, we can make a King model with 10 stars and $W_0=1.2$ by issuing the command:

In [ ]:
!makeking -n 10 -w 1.2

The down side (there's always a down side, right?) is that this is unwieldy, and we can't easily deal with the output. (We can get at it, just not very *easily*).

A better way to deal with this is to use python's `subprocess` module. This allows us to specify input and output of commands, among other things.  We import it as we have imported other modules in the past, and since I'm going to be using both `subprocess.Popen` and `subprocess.PIPE` quite a bit, and don't want to keep typing `subprocess` all the time, I'll import those two things individually.

In [ ]:
import subprocess
from subprocess import Popen, PIPE

To execute the same command as before (to generate a King model) I use `Popen`, thus:

In [ ]:
king = Popen(["makeking","-n", "5", "-w","5"], stdout=PIPE, stderr=PIPE) 

Notice there's no output (yet). This isn't just  because I've stored the results of the `Popen` call in a variable called king; it's also because even though the command has been built, it hasn't actually run yet. I'll get to that in a minute. Before we get to that, though, there are two more points I want to draw your attention to.

First, the command name and its arguments are elements of a list.  Python lists are comma separated collections of objects residing between square brackets.

The second thing I want to point out is the last two arguments of the `Popen` call. The two objects `stdout` and `stderr` are output streams for the `makeking` program. They stand for *standard output* and *standard error* respectively. In this case, `makeking` and the other programs that make up starlab send the current collection of stars being simulated to `stdout`, and logging information to `stderr`.  We'll exploit this to get data into and out of the starlab programs.

In [ ]:
results = king.communicate()

Stdout is contained in the first element of `results`:

In [ ]:
results[0]

Not very readable. But, if we send it through `print`, rather than just inspecting it, it cleans up a bit. 

In [ ]:
print results[0]

If we look closely at this, we can see that it contains some overall information about the distribution, and then the bulk of the output is devoted to giving masses, positions, and velocities for each of the stars. That's what we'll have to parse to do the plotting.

The other part of `results` is stderr; it contains other information about running the command. We can think of this as metadata, and it really ought to be stored with the rest of our results so that we have a record of what starlab thought at the time the data was created.

In [ ]:
print results[1]

### Adding Complexity

Most of the examples on the starlab website chain together a longish sequence of commands. For example, you often want to make a spatial distribution, then apply a mass function to it, then do some scaling, and add some binaries, and then integrate the whole thing, or generate statistics.  We can do that pretty easily using the framework I've outlined above. 

We start by creating a list of commands. Recall that each command is itself a list containing the command name and all of its arguments.  So we end up with a list of lists.

In [ ]:
cmds = []

cmds.append(["makeking", "-n", "500", "-w", "5", "-i",  "-u"])
cmds.append(["makemass", "-f", "2", "-l", "0.1,", "-u", "20"])
cmds.append(["makesecondary", "-f", "0.1", "-l", "0.25"])
cmds.append(["scale", "-m", "1", "-e", "-0.25", "-q", "0.5"]) 
cmds.append(["makebinary", "-l", "1", "-u", "10"])
cmds.append(["sys_stats", "-n"])

print cmds

The `Popen` command wouldn't know what to do with this list of lists, so we have to make a process for each piece. We want the output of each command to be the input for the next command (i.e., we apply `makemass` to the output of `makeking` in this example); we do this by connecting the `stdin` of each process to the `stdout` of the previous process.  We tell `Popen` that we're going to want to do this by using the special handle `PIPE`, below.

In [ ]:
procs = []
for index, cmd in enumerate(cmds):
    print index, cmd
    if index > 0:
        procs.append(Popen(cmd, stdout=PIPE, stderr=PIPE, stdin=procs[index-1].stdout))
    else:
        procs.append(Popen(cmd, stdout=PIPE, stderr=PIPE))
    inp = procs[-1].stdout
    

The `if` statement here just reflects the fact that our first command doesn't have any input, so we don't want to try to connect it to anything.  We get the output of the whole chain of commands by telling the last one to communicate.  It then requests its input from previous commands, and they're required to execute.

In [ ]:
result = procs[-1].communicate()
print "---------------- stdout ------------------"
print result[0]
print "---------------- stderr ------------------"
print result[1]

Note that in this case, `stdout` gives us nothing, while `sdterr` gives the statistics we asked for. 

We can do the same thing again, but instead of asking for statistics, we run kira to evolve the star cluster in time.

In [ ]:
cmds = []

cmds.append(["makeking", "-n", "500", "-w", "5", "-i",  "-u"])
cmds.append(["makemass", "-f", "2", "-l", "0.1,", "-u", "20"])
cmds.append(["makesecondary", "-f", "0.1", "-l", "0.25"])
cmds.append(["makebinary", "-l", "1", "-u", "10"])
cmds.append(["scale", "-m", "1", "-e", "-0.25", "-q", "0.5"]) 
cmds.append(["kira", "-t", "100", "-d", "1", "-D", "10", "-f", "0.3", "-n", "10", "-q", "0.5", "-G", "2", "-B"])


In [ ]:
procs = []
for index, cmd in enumerate(cmds):
    print index, cmd
    if index > 0:
        procs.append(Popen(cmd, stdout=PIPE, stderr=PIPE, stdin=procs[index-1].stdout))
    else:
        procs.append(Popen(cmd, stdout=PIPE, stderr=PIPE))
    inp = procs[-1].stdout
    
result = procs[-1].communicate()
print "---------------- stdout ------------------"
print result[1]
print "---------------- stderr ------------------"
print result[0]

Having typed the preceding block of code twice now, I'm sure I'm going to use it again ("Everything that happens once can never happen again. But everything that happens twice will surely happen a third time" -- Arab Proverb, according to Paulo Coelho) so I'll make a function out of it.

In [ ]:
def process_commands(cmds):
    procs = []
    for index, cmd in enumerate(cmds):
        if index > 0:
            procs.append(Popen(cmd, stdout=PIPE, stderr=PIPE, stdin=procs[index-1].stdout))
        else:
            procs.append(Popen(cmd, stdout=PIPE, stderr=PIPE))
        inp = procs[-1].stdout
    
    result = procs[-1].communicate()
    return result

I have left off the print statements since generally I want to do something with the result other than look at it.

### Dealing with the results

Now that we can generate some results, we'd like to be able to do something meaningful with them. The first thing we want to do is to plot a snapshot of the star field at a moment in time. To get to that point, we need to be able to transform the nearly human readable output format that comes from e.g. `makeking` or `makeplummer` into a list of positions, velocities, masses, and so on.

Starlab output is organized into "Stories".  Stories can consist of definitions, lines of text, and other stories.  In my mind, this maps reasonably well onto a python class, so that's what I'm going to do with it.

In [ ]:
import re

class Story:
    def __init__(self):
        self.story_lines = []
        self.story_vals = dict()
        self.story_subobjects = []
        self.kind = None
        return
    
    def __str__(self):
        return "%s, %d lines, %d values, %d subobjects" % (self.kind, len(self.story_lines), len(self.story_vals.keys()), len(self.story_subobjects))

    def process_line(self, line):
        # if we can tokenize into an equality, store in the dict, otherwise as a line.
        chunks = re.split('=', line)
#        print chunks, len(chunks)
        if len(chunks) == 2:
            self.story_vals[chunks[0].strip()] = chunks[1].strip()
        else:
            self.story_lines.append(line)
            

Stories begin with a line that looks like

`(Storyname`

and end with a line that looks like

`)Storyname`

To turn output lines into stories, we look at each line. When we see a line that starts a story, we launch in to "story parsing mode" and build a new story from that point until we hit the end of the story.  That action is taken care of in the `parse_lines` function below. The `parse_output` function is mostly just a wrapper so that the top level output can generate the right `parse_lines` call to begin with.

In [ ]:
def parse_output(results):
    """Move through a set of output from a starlab program, building a list of stories."""
    stories = []
    lines = re.split("\n", results)

    nextidx = -1
    
    for index, line in enumerate(lines):
        if index >= nextidx:
            storystart = re.match("^\((\w+)",line)
            if storystart:
                #print "in parse_output, calling parse_lines:", index, storystart.group(1)
                nextidx, newstory = parse_lines(lines, index+1, storystart.group(1))
                #print "in parse_output, back from parse_lines:", nextidx, str(newstory)
                stories.append(newstory)
    return stories

In [ ]:
def parse_lines(lines, startidx, name):
    """Recursively define stories from lines of output."""
    thestory = Story()
    thestory.kind = name
    nextidx = -1
    for index,line in enumerate(lines[startidx:]):
        if index >= nextidx-startidx:
            #print "%s Line is %s"%(name, line)
            storystart = re.match("^\((\w+)",line)
            storyend = re.match("\)%s"%name, line)
            if storyend: # we've hit the end of our story; get out and pass it back up
                endindex = index
                break
            elif storystart: # new story; start up a new parse_lines
                #print "in parse_lines, calling parse_lines:", startidx+index+1, storystart.group(1)
                nextidx, newstory = parse_lines(lines, startidx+index + 1, storystart.group(1))
                #print "back", nextidx, str(newstory)
                thestory.story_subobjects.append(newstory)
            else:
                thestory.process_line(line)
    return endindex+startidx+1, thestory

Let's see if it works. Generate a king model with 5 stars, and check the output.

In [ ]:
cmds = []
cmds.append(["makeking", "-n", "5", "-w", "5", "-i",  "-u"])
result = process_commands(cmds)
slist = parse_output(result[0])

How many stories do we get in our list?

In [ ]:
len(slist)

All right, what is it?

In [ ]:
print slist[0]

So we have a `Particle`, with 0 lines, 1 value, and 9 subobjects.  This `Particle` represents the cluster as a whole.  Let's look at the value.

In [ ]:
print slist[0].story_vals

This gives us the overall number of sub-particles.  What about the subobjects?

In [ ]:
for obj in slist[0].story_subobjects:
    print obj

The `Log`, `Dynamics`, `Hydro` and `Star` stories are substories of all `Particles`.  We additionally have `Particle` stories for the 5 particles in the cluster.  The values in the `Dynamics` and `Star` stories might be useful:

In [ ]:
print slist[0].story_subobjects[1]
print slist[0].story_subobjects[1].story_vals

In [ ]:
print slist[0].story_subobjects[3]
print slist[0].story_subobjects[3].story_vals

For plotting, we're only going to need the information stored in the `Dynamics` substory of each `Particle` substory.  These two functions extract that information and then make a plot.

In [ ]:
def extract_particle_dynamics(story):
    """ recursively extract dynamics objects to get masses, positions, and velocities.

    Quick and dirty."""
    particles = []
    if story.kind == "Dynamics":
        x, y, z = story.story_vals['r'].split(" ")
        vx, vy, vz = story.story_vals['v'].split(" ")
        m = story.story_vals['m']
        particles.append( (float(x), float(y), float(z), float(vx), float(vy), float(vz), float(m)) )
        #print "appending: ", particles[-1]
    elif len(story.story_subobjects) > 0:
        for obj in story.story_subobjects:
            particles.extend(extract_particle_dynamics(obj))

    return particles

def vis_story_3d(story_list):
    xvals = []
    yvals = []
    zvals = []
    vxs = []
    vys = []
    vzs = []
    masses = []
    
    # the way this is currently set up, we only keep the particle list from the last story
    for story in story_list:
        partlist = extract_particle_dynamics(story)
        
    for particle in partlist:
        xvals.append(particle[0])
        yvals.append(particle[1])
        zvals.append(particle[2])
        vxs.append(particle[3])
        vys.append(particle[4])
        vzs.append(particle[5])
        masses.append(particle[6])
        
    # now do the plot
    fig = plt.figure(figsize=(10,10))
    ax = fig.gca(projection='3d')
    # not super happy with the colors yet.
    ax.scatter(np.array(xvals), np.array(yvals), np.array(zvals), c=np.array(masses))
    #ax.plot(np.array(xvals), np.array(yvals), np.array(zvals), ".")

Before we can try it out, we need to make sure matplotlib is loaded, and ready to work in three dimensions.

In [ ]:
%pylab inline
from mpl_toolkits.mplot3d import axes3d

In [ ]:
vis_story_3d(slist)

In [ ]:
barking = Popen(["makeking","-n", "2000", "-w","5"], stdout=PIPE, stderr=PIPE) 
results = barking.communicate()
slist = parse_output(results[0])
vis_story_3d(slist)

In [ ]:
cmds = []
cmds.append(["makeking", "-n", "500", "-w", "5", "-i",  "-u"])
cmds.append(["makemass", "-f", "2", "-l", "0.1,", "-u", "20"])
result = process_commands(cmds)
slist = parse_output(result[0])
vis_story_3d(slist)

In [ ]:
cmds = []
cmds.append(["makeking", "-n", "500", "-w", "5", "-i",  "-u"])
cmds.append(["makemass", "-f", "2", "-l", "0.1,", "-u", "20"])
cmds.append(["kira", "-t", "10", "-d", "1", "-D", "2", "-f", "0.3", "-n", "10", "-q", "0.5", "-G", "2", "-B"])
result = process_commands(cmds)
slist = parse_output(result[0])
vis_story_3d(slist)

In [ ]:
len(slist[4].story_subobjects)

In [ ]:
cmds = []

cmds.append(["makeplummer", "-n", "500", "-i"])
cmds.append(["kira", "-t", "10", "-d", "1", "-D", "2", "-n", "10", "-q", "0.5", "-G", "2"])
procs = []
for index, cmd in enumerate(cmds):
    print index, cmd
    if index > 0:
        procs.append(Popen(cmd, stdout=PIPE, stderr=PIPE, stdin=procs[index-1].stdout))
    else:
        procs.append(Popen(cmd, stdout=PIPE, stderr=PIPE))
    inp = procs[-1].stdout
    
result = procs[-1].communicate()

In [ ]:
slist = parse_output(result[0])
vis_story_3d(slist)

In [ ]:
nparticles = len(extract_particle_dynamics(slist[-1]))
print nparticles

In [ ]:
remaining = []
nparticles = []

In [ ]:
def premain(startn):
    """Run a plummer model for 10 dynamical times and return the number of stars remaining."""
    from subprocess import Popen, PIPE
    from starlab import parse_output, extract_particle_dynamics
    
    print "running %d particles" % startn
    cmds = []

    cmds.append(["makeplummer", "-n", "%d"%startn, "-i"])
    cmds.append(["kira", "-t", "10", "-d", "1", "-D", "2", "-n", "10", "-q", "0.5", "-G", "2"])
    procs = []
    for index, cmd in enumerate(cmds):
        print index, cmd
        if index > 0:
            procs.append(Popen(cmd, stdout=PIPE, stderr=PIPE, stdin=procs[index-1].stdout))
        else:
            procs.append(Popen(cmd, stdout=PIPE, stderr=PIPE))
    inp = procs[-1].stdout
    
    result = procs[-1].communicate()
    slist = parse_output(result[0])
    return len(extract_particle_dynamics(slist[-1]))

In [ ]:
plt.plot(nparticles, np.array(remaining)/np.array(nparticles))

In [ ]:
fparts = [float(part) for part in remaining ]
fraction = np.array(fparts)/np.array(nparticles)
plt.plot(nparticles, fraction)

In [ ]:
from IPython.parallel import Client
rc = Client(profile='cluster')
len(rc.ids)

In [ ]:
v = rc.load_balanced_view()

In [ ]:
thepoints = [ 500 ] * 140
results = v.map_async(premain, thepoints)

In [ ]:
foo = results.get()

In [ ]:
import numpy as np
res = np.array(results)

In [ ]:
%pylab inline

In [ ]:
plt.hist(res, res.max()-res.min())

In [ ]:
res.min()

In [ ]:
res.max()

In [ ]:
thepoints = [ 500 ] * 10000
results = v.map(premain, thepoints, block=True)
res = np.array(results)
plt.hist(res, res.max()-res.min())

In [ ]:
premain(500)

In [ ]:
from IPython.parallel import Client
rc = Client(profile='cluster')
len(rc.ids)